# Exercises

## Data Acquisition

These exercises should go in a notebook or script named `wrangle`. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.



In [1]:
import pyspark
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/16 18:21:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read the case, department, and source data into their own spark dataframes.



In [2]:
df = case = spark.read.csv('case.csv', header=True, inferSchema=True)


In [3]:
depts = spark.read.csv('dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('source.csv', header=True, inferSchema=True)


In [4]:
df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')


In [5]:
df.show(5)

+---------+----------------+----------+----------------+----------------+------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
|source_id|   dept_division|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|   SLA_days|case_status|     request_address|council_district|           dept_name|standardized_dept_name|dept_subject_to_SLA|source_username|
+---------+----------------+----------+----------------+----------------+------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
| svcCRMLS|Field Operations|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        

Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both `csv` and `json` format, store these as `sources_csv` and `sources_json`



In [15]:
df.write.csv('sources_csv')


In [14]:
df.write.json('sources_json')

Inspect your folder structure. What do you notice?



Made folder with files

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.



In [6]:
df.dtypes

[('source_id', 'string'),
 ('dept_division', 'string'),
 ('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_username', 'string')]

- Make `council_district` string
- Make `case_late`and `case_closed` boolean
- Rename due date column
- Convert all dates to datetime from string

In [7]:
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')


In [8]:
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

In [9]:
df.show(5)

+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
|source_id|   dept_division|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|service_request_type|   SLA_days|case_status|     request_address|council_district|           dept_name|standardized_dept_name|dept_subject_to_SLA|source_username|
+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
| svcCRMLS|Field Operations|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-0

How old is the latest (in terms of days past SLA) currently open issue? 

In [16]:
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

In [18]:
df.createOrReplaceTempView('df')


In [19]:
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1856|
|         1856|
|         1856|
|         1855|
|         1853|
|         1849|
|         1849|
|         1848|
|         1847|
|         1847|
|         1843|
|         1842|
|         1842|
|         1842|
|         1839|
+-------------+



How long has the oldest (in terms of days since opened) currently opened issue been open?


In [20]:
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_opened_date) AS days_past_opened
FROM df
WHERE NOT case_closed
ORDER BY days_past_opened DESC
LIMIT 15
''').show()

+----------------+
|days_past_opened|
+----------------+
|            1872|
|            1872|
|            1871|
|            1870|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
|            1869|
+----------------+



How many Stray Animal cases are there?


In [21]:
df.filter(df.service_request_type == 'Stray Animal').count()


27361

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?



In [22]:
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

Convert the council_district column to a string column.



>See above

Extract the year from the case_closed_date column.


In [23]:
df.select('case_closed_date', year('case_closed_date')).show(5)


+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



Convert num_days_late from days to hours in new columns num_hours_late.



In [32]:
(
    df.withColumn('num_hours_late', round(df.num_days_late * 24))
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+--------------+
|      num_days_late|num_hours_late|
+-------------------+--------------+
| -998.5087616000001|      -23964.0|
|-2.0126041669999997|         -48.0|
|       -3.022337963|         -73.0|
|       -15.01148148|        -360.0|
|0.37216435200000003|           9.0|
|       -29.74398148|        -714.0|
|       -14.70673611|        -353.0|
|       -14.70662037|        -353.0|
|       -14.70662037|        -353.0|
|       -14.70649306|        -353.0|
|       -14.70649306|        -353.0|
|       -14.70636574|        -353.0|
|          -14.70625|        -353.0|
|       -14.70636574|        -353.0|
|       -14.70623843|        -353.0|
|-14.705891199999998|        -353.0|
|       -14.70600694|        -353.0|
|       -14.70576389|        -353.0|
|       -14.70576389|        -353.0|
|       -14.70564815|        -353.0|
+-------------------+--------------+
only showing top 20 rows



Join the case data with the source and department data.



>See above

Are there any cases that do not have a request source?



In [25]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



What are the top 10 service request types in terms of number of requests?



In [26]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)


+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



What are the top 10 service request types in terms of average days late?


In [33]:
(
    df.where('case_late')
    .groupBy('service_request_type')
    .agg(round(mean('num_days_late')).alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+-----------+-------+
|service_request_type                  |n_days_late|n_cases|
+--------------------------------------+-----------+-------+
|Zoning: Recycle Yard                  |211.0      |132    |
|Zoning: Junk Yards                    |200.0      |262    |
|Structure/Housing Maintenance         |190.0      |51     |
|Donation Container Enforcement        |171.0      |122    |
|Storage of Used Mattress              |164.0      |7      |
|Labeling for Used Mattress            |162.0      |7      |
|Record Keeping of Used Mattresses     |154.0      |7      |
|Signage Requied for Sale of Used Mattr|152.0      |12     |
|Traffic Signal Graffiti               |138.0      |4      |
|License Requied Used Mattress Sales   |129.0      |7      |
+--------------------------------------+-----------+-------+
only showing top 10 rows



Does number of days late depend on department?


In [34]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



How do number of days late depend on department and request type?


In [35]:
(
    df.filter("case_closed")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .where(col('days_late') > 0)
    .sort(desc("days_late"))
    .show(40, truncate=False)
)


+------------------------+----------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                    |days_late|n_cases|
+------------------------+----------------------------------------+---------+-------+
|DSD/Code Enforcement    |Zoning: Junk Yards                      |209.4    |174    |
|DSD/Code Enforcement    |Labeling for Used Mattress              |162.4    |7      |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses       |154.0    |7      |
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr  |151.6    |12     |
|DSD/Code Enforcement    |Storage of Used Mattress                |142.1    |8      |
|DSD/Code Enforcement    |Donation Container Enforcement          |141.3    |114    |
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |139.0    |141    |
|DSD/Code Enforcement    |License Requied Used Mattress Sales     |128.8    |7      |
|DSD/Code Enforcement    |Vendors                     

22/02/16 20:25:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 132823 ms exceeds timeout 120000 ms
22/02/16 20:25:13 WARN SparkContext: Killing executors is not supported by current scheduler.
